In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('simulations_day_output.csv', sep=';', header=None)
df.columns = ['simulation_key', 'missing_plan_cnt_first_delay', 'missing_plan_cnt_planned_time', 'whole_process_delay']
df['controller_name'] = df.simulation_key.str.split('-', expand=True)[0]
df['break_strategy'] = df.simulation_key.str.split('-', expand=True)[1]
df['experiment_name'] = df.controller_name + '-' + df.break_strategy
df['simulation_index'] = df.simulation_key.str.split('-', expand=True)[2].astype(int)
df['produced_in_planned_time'] = 1416 - df.missing_plan_cnt_planned_time
df['whole_process_delay'] = df.whole_process_delay.str.replace(',', '.').astype(float)
df['process_delay_minutes'] = df.whole_process_delay / 60
df['process_delay_hours'] = df.process_delay_minutes / 60
df.head()

,simulation_key,missing_plan_cnt_first_delay,missing_plan_cnt_planned_time,whole_process_delay,controller_name,break_strategy,experiment_name,simulation_index,produced_in_planned_time,process_delay_minutes,process_delay_hours
0,naive-skip_break-0,0,0,0.0,naive,skip_break,naive-skip_break,0,1416,0.0,0.0
1,naive-reorganization-0,0,0,0.0,naive,reorganization,naive-reorganization,0,1416,0.0,0.0
2,async-skip_break-0,0,0,0.0,async,skip_break,async-skip_break,0,1416,0.0,0.0
3,async-reorganization-0,0,0,0.0,async,reorganization,async-reorganization,0,1416,0.0,0.0
4,naive-skip_break-1,0,0,0.0,naive,skip_break,naive-skip_break,1,1416,0.0,0.0


In [3]:
df.groupby('experiment_name')[['produced_in_planned_time', 'process_delay_minutes']].describe()

produced_in_planned_time                               \
                                        count     mean         std     min   
experiment_name                                                              
async-reorganization                     50.0  1416.00    0.000000  1416.0   
async-skip_break                         50.0  1416.00    0.000000  1416.0   
naive-reorganization                     50.0  1364.60  113.063284  1074.0   
naive-skip_break                         50.0  1364.58  113.054274  1074.0   

                                                      process_delay_minutes  \
                          25%     50%     75%     max                 count   
experiment_name                                                               
async-reorganization  1416.00  1416.0  1416.0  1416.0                  50.0   
async-skip_break      1416.00  1416.0  1416.0  1416.0                  50.0   
naive-reorganization  1404.25  1414.0  1416.0  1416.0                  50.0   
naive-skip_break      1404.25  1414.0  1416.0  1416.0                  50.0   

                                                                      \
                           mean         std  min       25%       50%   
experiment_name                                                        
async-reorganization   0.000000    0.000000  0.0  0.000000  0.000000   
async-skip_break       0.000000    0.000000  0.0  0.000000  0.000000   
naive-reorganization  65.130490  139.830267  0.0  2.758292  6.584583   
naive-skip_break      65.236753  139.808801  0.0  2.996917  6.868917   

                                             
                            75%         max  
experiment_name                              
async-reorganization   0.000000    0.000000  
async-skip_break       0.000000    0.000000  
naive-reorganization  18.584917  429.901000  
naive-skip_break      18.593250  429.950167

In [19]:
df_summaries = pd.DataFrame(index=df.experiment_name.unique(),
                            columns=['perc_complete_plans', 'perc_zero_delay_plans',
                                    'avg_missing_steps', 'avg_accumulated_delay',
                                    'missing_steps_90_percentile', 'missing_steps_95_percentile', 'missing_steps_99_percentile',
                                    'missing_steps_70_percentile', 'missing_steps_80_percentile'])

for group_name, df_group in df.groupby('experiment_name'):
    df_summaries.loc[group_name, 'perc_complete_plans'] = df_group[df_group.missing_plan_cnt_planned_time == 0].shape[0] / df_group.shape[0] * 100
    df_summaries.loc[group_name, 'perc_complete_plans'] = str(df_summaries.loc[group_name, 'perc_complete_plans']) + ' %'
    df_summaries.loc[group_name, 'perc_zero_delay_plans'] = df_group[df_group.whole_process_delay == 0].shape[0] / df_group.shape[0] * 100
    df_summaries.loc[group_name, 'perc_zero_delay_plans'] = str(df_summaries.loc[group_name, 'perc_zero_delay_plans']) + ' %'
    df_summaries.loc[group_name, 'avg_missing_steps'] = df_group.missing_plan_cnt_planned_time.mean()
    df_summaries.loc[group_name, 'avg_accumulated_delay'] = df_group.process_delay_minutes.mean()
    df_summaries.loc[group_name, 'missing_steps_90_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.9)
    df_summaries.loc[group_name, 'missing_steps_95_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.95)
    df_summaries.loc[group_name, 'missing_steps_99_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.99)
    df_summaries.loc[group_name, 'missing_steps_70_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.70)
    df_summaries.loc[group_name, 'missing_steps_80_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.80)

df_summaries

,perc_complete_plans,perc_zero_delay_plans,avg_missing_steps,avg_accumulated_delay,missing_steps_90_percentile,missing_steps_95_percentile,missing_steps_99_percentile,missing_steps_70_percentile,missing_steps_80_percentile
naive-skip_break,42.0 %,10.0 %,51.42,65.2368,295,341.55,342,10,28.6
naive-reorganization,42.0 %,12.0 %,51.4,65.1305,295,341.55,342,10,28.6
async-skip_break,100.0 %,100.0 %,0,0,0,0,0,0,0
async-reorganization,100.0 %,100.0 %,0,0,0,0,0,0,0


| Experiment | Plánovaný počet kusů | Procentuální počet splněných plánů | Procentuální počet plánů bez zpoždění | Průměrný počet vyprodukovaných kusů za plánované období 1-|
| --- | --- | --- | --- | --- |
| Naivní bez přeskládávání | 1416 | .608 | 0.161 | 1391.87 | 29.90

In [6]:
px.box(df, y='missing_plan_cnt_planned_time', color='experiment_name', title='Počet chybějících aut z plánu')

In [7]:
px.box(df, y='process_delay_minutes', color='experiment_name', title='Akumulovaný čas prostoje v minutách')

Kontrola zda je zaznam bez delay ktery se uspesne nedokoncil

In [8]:
df[(df.missing_plan_cnt_planned_time > 0) & (df.whole_process_delay == 0)]

,simulation_key,missing_plan_cnt_first_delay,missing_plan_cnt_planned_time,whole_process_delay,controller_name,break_strategy,experiment_name,simulation_index,produced_in_planned_time,process_delay_minutes,process_delay_hours


Maximalni delay uspesne vyrobene cele vyrobni rady

In [9]:
df[(df.missing_plan_cnt_planned_time == 0)].sort_values('whole_process_delay', ascending=False).head()

,simulation_key,missing_plan_cnt_first_delay,missing_plan_cnt_planned_time,whole_process_delay,controller_name,break_strategy,experiment_name,simulation_index,produced_in_planned_time,process_delay_minutes,process_delay_hours
152,naive-skip_break-38,1373,0,302.03,naive,skip_break,naive-skip_break,38,1416,5.033833,0.083897
153,naive-reorganization-38,1373,0,302.03,naive,reorganization,naive-reorganization,38,1416,5.033833,0.083897
128,naive-skip_break-32,1369,0,280.93,naive,skip_break,naive-skip_break,32,1416,4.682167,0.078036
144,naive-skip_break-36,1372,0,280.17,naive,skip_break,naive-skip_break,36,1416,4.669500,0.077825
129,naive-reorganization-32,1338,0,275.16,naive,reorganization,naive-reorganization,32,1416,4.586000,0.076433


In [10]:
px.box(df[(df.missing_plan_cnt_planned_time == 0)], y='whole_process_delay', title='Délka prostoje v sekundách pro splněné plány')

In [11]:
px.box(df, y='missing_plan_cnt_first_delay', color='experiment_name', title='Počet chybějících plánovaných kroku při prvním prostoji')